In [24]:
import enum
from langchain_core.messages.base import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import logging as log
import os
import pprint

log.basicConfig(level=log.INFO, format='%(asctime)s [%(levelname)5s] %(message)s',datefmt='%H:%M:%S')

In [25]:
class FinishReason(enum.Enum):
    Stop=0,
    LengthExceeded=1

def parseFinishReason( message: BaseMessage):
    finishReason= message.response_metadata['finish_reason']
    if 'STOP'==finishReason:
        return FinishReason.Stop        
    elif 'LENGTH'==finishReason:       
        return FinishReason.LengthExceeded
    raise ValueError(f'Invalid finish reason:{s}')

In [26]:
API_KEY='GOOGLE_API_KEY'
MODEL_NAME='gemini-1.5-pro'
assert API_KEY in os.environ 
assert len( str(API_KEY).strip() ) > 0
log.info(f'Got envvar:{API_KEY}.')

17:00:26 [ INFO] Got envvar:GOOGLE_API_KEY.


In [27]:
model = ChatGoogleGenerativeAI(model=MODEL_NAME, 
                               max_output_tokens=100,
                               temperature=0.0)

template= ChatPromptTemplate([('system','You are a helpful assistant.'),
                             ('human','{request}')])
parser= StrOutputParser()

chain = template | model | parser

In [28]:
result= chain.invoke({ 'request':"""
Generate a table with the 5 most popular pharmaceutical companies and their foundation years. 
The response should include only the table, with no additional text. 
Use the following example format:
---
Company | Foundation Year
Microsoft | 1975
Apple | 1976
Google | 1998
Amazon | 1994
Meta | 2004
---"""})

In [29]:
print(result)

```
Company | Foundation Year
------- | --------
Johnson & Johnson | 1886
Pfizer | 1849
Roche | 1896
Novartis | 1996
AbbVie | 2013
```


## Set up the judge

In [35]:
templateJudge= ChatPromptTemplate([('system',"""You are a strict judge. Evaluate inputs based on the given criteria and provide only the required response.
Your task is to verify if a given table matches the exact format and structure of a provided example table.
Here's an example of the format that the table should have:
---
Company | Foundation Year
Microsoft | 1975
Apple | 1976
Google | 1998
Amazon | 1994
Meta | 2004
---"""),
                                   ('human',""" 
Table to Check:
{table_to_check}

Instructions:
- The table to check should match the structure, headers, and format of the Example Table exactly.
- The column names must be "Company" and "Foundation Year".
- The values in each row should have the company names and their corresponding foundation years.
- If the given table matches the example table in all these aspects, write "Yes".
- Write "No" if there are any differences in structure, headers, or if any company/year is missing or incorrect.

Only respond with "Yes" or "No".                                   
""")])
parserJudge= StrOutputParser()

chainJudge = templateJudge | model | parserJudge

In [36]:
resultJudge= chainJudge.invoke({ 'table_to_check':result})

In [37]:
print(resultJudge)

Yes


### Use a badly formatted table.

In [38]:
badly_formatted_table = """
Company | Foundation Year

Microsoft | 1975

Apple | 1976
Google | 1998
Amazon | 1994

Meta | 2004
"""

In [39]:
resultJudge= chainJudge.invoke({ 'table_to_check':badly_formatted_table})
print(resultJudge)

No


## Judging the Company Type

In [43]:
templateJudge= ChatPromptTemplate([('system',"""You are a strict judge. Evaluate inputs based on the given criteria and provide only the required response."""),
                                   ('human',"""Your task is to verify if a given table contains only companies from the pharmaceutical industry.
Here's an example of the format that the table should have:
---
Company | Foundation Year
Microsoft | 1975
Apple | 1976
Google | 1998
Amazon | 1994
Meta | 2004
---       

Here's the table to check:
{table_to_check}

Instructions:
- The table to check should match the structure, headers, and format of the Example Table exactly.
- The column names must be "Company" and "Foundation Year".
- Evaluate whether each company in the table is part of the pharmaceutical sector.
- If all companies in the table are from the pharmaceutical industry, write "Yes".
- Write "No" if any company in the table is not from the pharmaceutical industry.

Only respond with "Yes" or "No", and nothing else.                                   
""")])
parserJudge= StrOutputParser()

chainJudge = templateJudge | model | parserJudge

In [44]:
resultJudge= chainJudge.invoke({ 'table_to_check':result})
print(resultJudge)

Yes


In [45]:
resultJudge= chainJudge.invoke({ 'table_to_check':badly_formatted_table})
print(resultJudge)

No
